In [1]:
import math
import torch
import random
from torch import nn
from torch.autograd import Variable
from torch.optim import Adam
import torch.nn.functional as F
import torchtext
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
from torch.nn.utils import clip_grad_norm
import spacy

In [2]:
def sequence_to_text(sequence, field):
    pad = field.vocab.stoi['<pad>']
    return " ".join([field.vocab.itos[int(i)] for i in sequence if int(i) != pad])

In [3]:
def load_dataset(batch_size, device=0):

    spacy_de = spacy.load('de')
    spacy_en = spacy.load('en')

    def tokenize_de(text):
        return [tok.text for tok in spacy_de.tokenizer(text)]

    def tokenize_en(text):
        return [tok.text for tok in spacy_en.tokenizer(text)]

    DE = Field(tokenize=tokenize_de, eos_token='<eos>')
    EN = Field(tokenize=tokenize_en, eos_token='<eos>')

    train, val, test = Multi30k.splits(exts=('.de', '.en'), fields=(DE, EN))

    DE.build_vocab(train.src)
    EN.build_vocab(train.trg)

    train_iter, val_iter, test_iter = BucketIterator.splits(
        (train, val, test), batch_size=batch_size, device=device, repeat=False)
    return train_iter, val_iter, test_iter, DE, EN

In [4]:
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embed_dim, hidden_dim,
                 n_layers=1, dropout=0.5):
        super(Encoder, self).__init__()
        self.embed = nn.Embedding(src_vocab_size, embed_dim, padding_idx=1)
        self.gru = nn.GRU(embed_dim, hidden_dim // 2, n_layers,
                          dropout=dropout, bidirectional=True)

    def forward(self, src, hidden=None):
        embedded = self.embed(src)  # (batch_size, seq_len, embed_dim)
        encoder_out, encoder_hidden = self.gru(
            embedded, hidden)  # (batch, seq_len, hidden_dim*2)
        return encoder_out, encoder_hidden

## Derive context vector from encoder out

In [5]:
class Attention(nn.Module):
    """
    General Attention from Effective Approaches to Attention-based Neural Machine Translation
    https://arxiv.org/pdf/1508.04025.pdf
    """
    def __init__(self, dim):
        super(Attention, self).__init__()
        self.W = nn.Linear(dim, dim, bias=False)

    def score(self, decoder_hidden, encoder_out):
        # linear transform encoder out (seq, batch, dim)
        encoder_out = self.W(encoder_out)
        # (batch, seq, dim) | (2, 15, 50)
        encoder_out = encoder_out.permute(1, 0, 2)
        # (2, 15, 50) @ (2, 50, 1)
        return encoder_out @ decoder_hidden.permute(1, 2, 0)

    def forward(self, decoder_hidden, encoder_out):
        energies = self.score(decoder_hidden, encoder_out)
        attention_weights = F.softmax(energies, dim=0)
        context = encoder_out.permute(
            1, 2, 0) @ attention_weights  # (2, 50, 15) @ (2, 15, 1)
        context = context.permute(2, 0, 1)  # (seq, batch, dim)
        return context, attention_weights

In [6]:
class Decoder(nn.Module):
    def __init__(self, trg_vocab_size, embed_dim, hidden_dim,
                 n_layers=1, dropout=0.5):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.embed = nn.Embedding(trg_vocab_size, embed_dim)
        self.attention = Attention(hidden_dim)
        self.gru = nn.GRU(embed_dim + hidden_dim, hidden_dim, n_layers,
                          dropout=dropout)
        self.out = nn.Linear(hidden_dim, trg_vocab_size)

    def init_hidden(self, encoder_hidden):
        """
        reshape (n_layers * direction, batch_size, dim) bidirectional encoder hidden to
        (layers, batch_size, dim*n_layers) hidden.
        """
        return encoder_hidden.view(-1, encoder_hidden.size(1), encoder_hidden.size(2) * 2)
    
    def forward(self, trg, encoder_out, decoder_hidden):
        """
        decode's once
        """
        embedded = self.embed(trg).unsqueeze(0)  # (1, batch, embed_dim)
        context, attention_weights = self.attention(
            decoder_hidden, encoder_out)  # 1, 1, 50 (seq, batch, hidden_dim)
        rnn_output, decoder_hidden = self.gru(torch.cat([embedded, context], dim=2),
                                              decoder_hidden)
        output = self.out(rnn_output)
        return output, decoder_hidden, attention_weights

In [7]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg=None, teacher_forcing_ratio=0.5, max_len=50):
        """
        feed targets and teacher_forcing_ratio at training time, set max_len at test time
        """
        max_len = len(trg) if trg is not None else None
        encoder_out, encoder_hidden = self.encoder(src)
        decoder_hidden = self.decoder.init_hidden(encoder_hidden)
        outputs = []
        attention_weights = []
        output = torch.zeros_like(src[0]) + 2  # initial output <sos> index
        for t in range(max_len):
            output, decoder_hidden, attn_weights = self.decoder(
                    output, encoder_out, decoder_hidden)
            outputs.append(output)
            attention_weights.append(attn_weights)
            is_teacher = random.random() < teacher_forcing_ratio
            top1 = output.data.max(2)[1].squeeze()
            output = Variable(trg[t].data if is_teacher else top1)
        return torch.cat(outputs), torch.cat(attention_weights)

In [8]:
hidden_dim = 512
embed_dim = 256
n_layers = 2
dropout=0.5

## Train

In [9]:
train_iter, val_iter, test_iter, DE, EN = load_dataset(batch_size=32, device=0)

encoder = Encoder(src_vocab_size=len(DE.vocab),
                  embed_dim=embed_dim, hidden_dim=hidden_dim,
                  n_layers=1, dropout=dropout)
decoder = Decoder(trg_vocab_size=len(EN.vocab),
                  embed_dim=embed_dim, hidden_dim=hidden_dim,
                  n_layers=1, dropout=dropout)
seq2seq = Seq2Seq(encoder, decoder)

In [10]:
seq2seq.cuda()

Seq2Seq(
  (encoder): Encoder(
    (embed): Embedding(19205, 256, padding_idx=1)
    (gru): GRU(256, 256, dropout=0.5, bidirectional=True)
  )
  (decoder): Decoder(
    (embed): Embedding(10839, 256)
    (attention): Attention(
      (W): Linear(in_features=512, out_features=512, bias=False)
    )
    (gru): GRU(768, 512, dropout=0.5)
    (out): Linear(in_features=512, out_features=10839, bias=True)
  )
)

In [11]:
def evaluate(model, val_iter, batch_size=32, generate_interval=100):
    model.eval()
    total_loss = 0
    fields = val_iter.dataset.fields
    for b, batch in enumerate(val_iter):
        if batch.trg.size(1) != batch_size: # ignore the last batch
            continue
        out, attention = model(batch.src, batch.trg, teacher_forcing_ratio=0)
        loss = F.cross_entropy(out.view(-1, out.size(2)),
                               batch.trg.view(-1), ignore_index=2)
        total_loss += loss.data[0]
        if b % generate_interval == 0:
            print('source: ', sequence_to_text(batch.src[:, 0], fields['src']))
            probs, pred = F.softmax(out, dim=-1).topk(1)
            print('Prediction: ', sequence_to_text(pred[:, 0], fields['trg']))
            print('Target: ', sequence_to_text(batch.trg[:, 0], fields['trg']))
            print('\n ')
    return total_loss / len(val_iter)


def train(model, optimizer, train_iter, val_iter,
          batch_size, num_epochs, teacher_forcing_ratio=0.5):
    
    model.train()
    total_loss = 0
    for epoch in range(num_epochs):
        for b, batch in enumerate(train_iter):
            if batch.trg.size(1) != 32: # ignore the last batch
                continue
            optimizer.zero_grad()
            out, attention = model(batch.src, batch.trg, teacher_forcing_ratio)
            loss = F.cross_entropy(out.view(-1, out.size(2)),
                                   batch.trg.view(-1), ignore_index=2)
            loss.backward()
            clip_grad_norm(model.parameters(), 10.0, norm_type=2) # prevent exploding grads
            optimizer.step()
            total_loss += loss.data[0]changes are handled at the language

            if b % 100 == 0 and b != 0:
                total_loss = total_loss / 100
                print("[%d][loss:%5.2f][pp:%5.2f]" %
                      (b, total_loss, math.exp(total_loss)))
                total_losstotal_loss = 0
        print(evaluate(model, val_iter, batch_size))

In [12]:
optimizer = Adam(seq2seq.parameters())

In [13]:
train(seq2seq, optimizer, train_iter, val_iter, batch_size=32, num_epochs=5)

[100][loss: 5.42][pp:226.49]
[200][loss: 4.83][pp:124.83]
[300][loss: 4.78][pp:119.21]
[400][loss: 4.73][pp:113.19]
[500][loss: 4.66][pp:105.73]
[600][loss: 4.61][pp:100.91]
[700][loss: 4.58][pp:97.99]
[800][loss: 4.56][pp:95.36]
[900][loss: 4.51][pp:91.02]


/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


source:  <sos> Die Dame trägt vor dem Spiegel farbigen Lippenstift auf ihre Lippen auf . <eos>
Prediction:  A A woman is a a a . <eos>
Target:  <sos> The lady is putting colored lipstick on her lips in the mirror . <eos>

 
source:  <sos> Drei Frauen auf der Straße , mit Bäumen im Hintergrund . <eos>
Prediction:  A A man in a a shirt is a a . <eos>
Target:  <sos> Three women down the street with trees in the background . <eos>

 
source:  <sos> Fünf Männer unterhalten sich in einem Raum mit Computern . <eos>
Prediction:  A A man is a a a . <eos>
Target:  <sos> Five men are talking in a room that has computers in it . <eos>

 
source:  <sos> Ein Mann sitzt auf einem Sims und liest Zeitung . <eos>
Prediction:  A A man and a and a and a a .
Target:  <sos> Man sitting on ledge reading the paper . <eos>

 
source:  <sos> Vier Kinder sind im Freien in einer Reihe mit durchgedrücktem Rücken . <eos>
Prediction:  A A woman is a a a . <eos>
Target:  <sos> Four children are outside in a line all 

/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  """


[100][loss: 4.70][pp:110.28]
[200][loss: 4.46][pp:86.63]
[300][loss: 4.41][pp:82.56]
[400][loss: 4.36][pp:78.50]
[500][loss: 4.35][pp:77.35]
[600][loss: 4.36][pp:78.58]
[700][loss: 4.38][pp:79.84]
[800][loss: 4.43][pp:83.95]
[900][loss: 4.48][pp:88.14]
source:  <sos> Die Feuerwehr ist vor Ort , um bei einem Unfall Hilfe zu leisten . <eos>
Prediction:  A A woman are in a and . <eos>
Target:  <sos> The Fire Department is on the scene to assist in an accident . <eos>

 
source:  <sos> Eine Frau mit weißem T-Shirt , weißen Shorts und Sonnenbrille . <eos>
Prediction:  A A man in a a . <eos>
Target:  <sos> A woman wearing a white t - shirt , white shorts , and sunglasses . <eos>

 
source:  <sos> Zwei Kinder , die Jeans tragen , gehen auf einer Landebahn . <eos>
Prediction:  A A man in a a . <eos>
Target:  <sos> Two children wearing denim walk on a runway . <eos>

 
source:  <sos> Zwei Männer in Karateanzügen kämpfen mit Kampfstöcken . <eos>
Prediction:  A A man in a a a a . <eos>
Target:  <

source:  <sos> Ein Vater wirft seine Tochter in einen Oceanside-Pool . <eos>
Prediction:  A A man is a a a a . <eos>
Target:  <sos> A father launches his daughter in an Oceanside pool . <eos>

 
source:  <sos> Eine Gruppe von fünf Kindern in Winterkleidung spielt im Schnee . <eos>
Prediction:  A A group of people are in a
Target:  <sos> A group of 5 kids are dressed in winter apparel playing in the snow . <eos>

 
source:  <sos> Ein kleiner Junge trifft andere Kinder auf Fahrrädern . <eos>
Prediction:  A A man in a blue shirt is a a . <eos>
Target:  <sos> Young boy encounters other children , also riding bikes . <eos>

 
source:  <sos> Eine Frau mit dunklen lockigen Haaren und einem dunklen Kleid singt und hält eine Gitarre . <eos>
Prediction:  A A woman in a and and a a a a a a . <eos>
Target:  <sos> A woman with dark curly hair and a dark dress is singing and holding a guitar . <eos>

 
source:  <sos> Eine kleine Person hält eine gelbe Flagge mit einem roten Logo . <eos>
Prediction: 

In [14]:
train(seq2seq, optimizer, train_iter, val_iter, batch_size=32, num_epochs=5)

[100][loss: 4.09][pp:59.48]
[200][loss: 4.14][pp:62.64]
[300][loss: 4.14][pp:62.88]
[400][loss: 4.05][pp:57.28]
[500][loss: 4.05][pp:57.43]
[600][loss: 4.10][pp:60.29]
[700][loss: 4.09][pp:59.48]
[800][loss: 4.08][pp:59.16]


/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


source:  <sos> Das ist unsere Frühstückszeit und wir posieren für das Foto . <eos>
Prediction:  A A woman and a and and a is sitting on a bench . <eos>
Target:  <sos> This is our break fast time and we are posing for the photo . <eos>

 
source:  <sos> Eine Frau mit Zöpfen kniet und stellt an einer Maschine eine Schnur ein . <eos>
Prediction:  A A woman in a and white shirt is a a <eos>
Target:  <sos> A woman with braids is kneeling down and adjusting string on a machine <eos>

 
source:  <sos> Ein Kind versucht , durch Laufen auf eine Skateboard-Bahn zu klettern . <eos>
Prediction:  A A woman in a a shirt is a a . <eos>
Target:  <sos> A kid trying to climb up running through a skateboard slope . <eos>

 
source:  <sos> Eine Frau mit Kopfbedeckung verkauft etwas an eine ältere Frau mit weißem Haar . <eos>
Prediction:  A A woman in a a shirt is a a . <eos>
Target:  <sos> A woman wearing a hat selling something to an older women with white hair . <eos>

 
source:  <sos> Ein Footballspiel

/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  """


[100][loss: 8.21][pp:3679.13]
[200][loss: 4.01][pp:55.27]
[300][loss: 4.03][pp:56.27]
[400][loss: 4.07][pp:58.84]
[500][loss: 4.07][pp:58.45]
[600][loss: 4.08][pp:58.93]
[700][loss: 4.02][pp:55.74]
[800][loss: 4.07][pp:58.77]
[900][loss: 4.07][pp:58.52]
source:  <sos> Ein Mann tanzt vor einer Musikgruppe . <eos>
Prediction:  A A man in a a shirt is a a . <eos>
Target:  <sos> A man is dancing in front of a band . <eos>

 
source:  <sos> Ein Mann spielt bei einem Livekonzert E-Gitarre . <eos>
Prediction:  A A man in a a shirt is a a .
Target:  <sos> Man playing electric guitar live during a concert . <eos>

 
source:  <sos> Eine Frau in schwarzem Minirock fährt auf einem orangefarbenen Fahrrad . <eos>
Prediction:  A A woman in a a a a a a . <eos>
Target:  <sos> A woman in a black miniskirt riding an orange bike . <eos>

 
source:  <sos> Zwei Bauarbeiter , von denen einer ein gelbes Metallstück hält und der andere Werkzeug . <eos>
Prediction:  A Two men are in a and a a . <eos>
Target:  <

source:  <sos> Ein Mann mit Hut dreht sich um und schaut sich Karten an . <eos>
Prediction:  A A man with a backpack is walking down a street . <eos>
Target:  <sos> A man with a hat is turned around looking at cards . <eos>

 
source:  <sos> Eine Gruppe von Menschen zieht mehrere Lagen Kleidung für den kalten Winter an . <eos>
Prediction:  A A man is is the camera of a the air . <eos>
Target:  <sos> A group of people putting on layers of clothes for the cold winter . <eos>

 
source:  <sos> Ein Mann in einem braunen Oberteil auf einer sonnigen Straße und einer Menschenmenge im Hintergrund . <eos>
Prediction:  A A man in a a shirt and a a a a a a a a
Target:  <sos> Man in brown shirt on sunny street , a crowd in background <eos>

 
source:  <sos> Zwei Taucher unter dem Wasser haben Spaß . <eos>
Prediction:  A A men are playing on a . <eos>
Target:  <sos> Two scuba divers underwater , enjoying their time . <eos>

 
source:  <sos> Ein kleiner Junge schwingt einen Golfschläger vor einem of

In [16]:
train(seq2seq, optimizer, train_iter, val_iter, batch_size=32, num_epochs=10)

[100][loss: 3.93][pp:51.07]
[200][loss: 3.96][pp:52.20]
[300][loss: 3.92][pp:50.45]
[400][loss: 3.95][pp:51.70]
[500][loss: 3.91][pp:49.65]
[600][loss: 3.98][pp:53.42]
[700][loss: 3.96][pp:52.60]
[800][loss: 3.94][pp:51.55]
[900][loss: 4.03][pp:56.45]


/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


source:  <sos> Ein Tänzer hebt seine Partnerin auf seine Schultern . <eos>
Prediction:  A A dog is . <eos>
Target:  <sos> A male dancer in beige lifts his female companion up on his shoulder , as she soars above the crowd . <eos>

 
source:  <sos> Eine Gruppe Leute mit Handschuhen arbeitet an großen bunten Quadraten . <eos>
Prediction:  A A woman of people are a a . . <eos>
Target:  <sos> A group of people with gloves on working on large colorful squares . <eos>

 
source:  <sos> Der Kerl im dunklen Anzug auf dem Podium zeigt auf den Bildschirm mit der Powerpoint-Präsentation . <eos>
Prediction:  A A woman is a a on a sidewalk of a large building . <eos>
Target:  <sos> The guy in the dark suit at the podium is pointing to the powerpoint screen . <eos>

 
source:  <sos> Menschen sitzen an langen Tischen , alle in dieselbe Richtung gewandt , manche von ihnen schreiben und manche schauen . <eos>
Prediction:  A Two women are in a and and a and a . . <eos>
Target:  <sos> People seated at lo

/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  """


[100][loss: 4.10][pp:60.26]
[200][loss: 3.81][pp:45.29]
[300][loss: 3.89][pp:48.89]
[400][loss: 3.90][pp:49.62]
[500][loss: 3.88][pp:48.35]
[600][loss: 3.99][pp:53.93]
[700][loss: 3.93][pp:51.07]
[800][loss: 3.93][pp:50.67]
source:  <sos> Mann fegt mit Zweigbündel Unrat von der Straße . <eos>
Prediction:  A A man in a a shirt is a . <eos>
Target:  <sos> Man sweeping debris from road with bundle of twigs . <eos>

 
source:  <sos> Zwei Männer fechten einen Kampf in einem Boxring aus . <eos>
Prediction:  A A man men in a a a a . <eos> .
Target:  <sos> Two men are duking it out in a boxing ring . <eos>

 
source:  <sos> Ein äteres Paar geht in den Abendstunden am Häuserblock entlang . <eos>
Prediction:  Two A man in a a shirt is a the . . <eos>
Target:  <sos> An elderly couple is walking down the block during the evening hour . <eos>

 
source:  <sos> Eine Frau in kariertem Rock und schwarzem Oberteil spielt auf einer elektrischen Geige . <eos>
Prediction:  A A woman and a man sitting on a

source:  <sos> Eine sehr glückliche Person geht zu Fuß durch den Wald , während die Blätter umher wehen . <eos>
Prediction:  A A woman is her toenails while the man is the the air . <eos>
Target:  <sos> A very happy person walking in the woods with the leaves blowing . <eos>

 
source:  <sos> Ein Mann grillt mit der Hilfe seiner Frau auf einem Propangasbrenner Steaks . <eos>
Prediction:  A A man in a a shirt is a a a . . <eos>
Target:  <sos> A man and his female assistant use a propane torch to grill steaks . <eos>

 
source:  <sos> Eine Gruppe von Menschen steht auf einer Straße mit hoch aufragenden Gebäuden auf beiden Seiten . <eos>
Prediction:  A A man is on the sidewalk of a group of people . <eos>
Target:  <sos> A group of people is standing on a street with towering buildings to each side of them . <eos>

 
source:  <sos> Der Spieler tritt einen Fußball auf dem Rasen . <eos>
Prediction:  A A man is climbing a rock . <eos>
Target:  <sos> The player is kicking a soccer ball on the 

source:  <sos> Drei weibliche Personen hängen , und die anderen drei weiblichen Personen gehen weg . <eos>
Prediction:  A A man in a white hat is sitting on a bench . <eos>
Target:  <sos> Three females are hanging while the other three females are walking away . <eos>

 
source:  <sos> Ein junges spanisches Mädchen springt vom betonierten Hofteiler auf den Bürgersteig . <eos>
Prediction:  A A surfer is the street on the street . <eos>
Target:  <sos> Young spanish girl jumps off the concrete yard divider onto the sidewalk . <eos>

 
source:  <sos> Ein Mann hält eine Metallstange in einen Lavastrom . <eos>
Prediction:  A A man is a a a . <eos> <eos>
Target:  <sos> A man sticking a metal rod into a river of lava . <eos>

 
143.98441080749035
[100][loss: 4.01][pp:55.31]
[200][loss: 3.78][pp:43.66]
[300][loss: 3.83][pp:46.02]
[400][loss: 3.89][pp:48.75]
[500][loss: 3.85][pp:46.86]
[600][loss: 3.82][pp:45.57]
[700][loss: 3.83][pp:46.15]
[800][loss: 3.77][pp:43.25]
[900][loss: 3.85][pp:47.07]

In [15]:
%pdb

Automatic pdb calling has been turned ON


In [17]:
train(seq2seq, optimizer, train_iter, val_iter, batch_size=32, num_epochs=20)

[100][loss: 3.80][pp:44.64]
[200][loss: 3.83][pp:46.22]
[300][loss: 3.90][pp:49.45]
[400][loss: 3.82][pp:45.76]
[500][loss: 3.90][pp:49.50]
[600][loss: 3.79][pp:44.18]
[700][loss: 3.91][pp:50.05]
[800][loss: 3.80][pp:44.74]
[900][loss: 3.85][pp:47.13]


/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


source:  <sos> Ein Wanderer entspannt sich auf einem Lehmhügel . <eos>
Prediction:  A A man is a a a a . .
Target:  <sos> A hiker relaxing on a clay hillside . <eos>

 
source:  <sos> Ein kleiner Junge sitzt in einem Auto in einem Fahrgeschäft . <eos>
Prediction:  A A black in a a of a a a a . <eos> <eos>
Target:  <sos> A small boy is sitting in a car at a ride . <eos>

 
source:  <sos> Frauen stehen in einem Kasino vor einer ATM-Maschine . <eos>
Prediction:  A A woman in a a shirt is sitting on a a a of a
Target:  <sos> Women standing in front of a ATM machine at a casino . <eos>

 
source:  <sos> Zwei Männer , einer in einem blauen Oberteil und blauen Hosen , der andere in einem lila Oberteil und schwarzen Hosen , stellen Töpferwaren her . <eos>
Prediction:  Two Two men are in a a , one in a , one in the background . <eos>
Target:  <sos> Two men , one dressed in blue shirt and pants and one dressed in purple shirt and black pants are creating pottery . <eos>

 
source:  <sos> Eine As

/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  """


[100][loss: 3.95][pp:51.69]
[200][loss: 3.77][pp:43.39]
[300][loss: 3.83][pp:45.93]
[400][loss: 3.82][pp:45.59]
[500][loss: 3.81][pp:45.10]
[600][loss: 3.84][pp:46.39]
[700][loss: 3.74][pp:42.14]
[800][loss: 3.82][pp:45.39]
[900][loss: 3.81][pp:45.33]
source:  <sos> Ein Footballteam läuft los , als der Kicker den Football mit dem Fuß berührt . <eos>
Prediction:  A A young girl in a pink shirt is in a of . <eos> <eos>
Target:  <sos> A football team is running as the kicker touches the football with his foot . <eos>

 
source:  <sos> Sie fahren mit dem Fahrrad durch die Straßen . <eos>
Prediction:  A The boy is is the the ball . <eos>
Target:  <sos> They are biking in the streets . <eos>

 
source:  <sos> Ein junger Unternehmer hält mit zwei MacBooks eine Präsentation . <eos>
Prediction:  A A man in a red shirt is a a . <eos>
Target:  <sos> Young entrepreneur giving a presentation using two mac book laptops . <eos>

 
source:  <sos> Ein dicker brauner Hund steht über einem kleinen weißen

source:  <sos> Ein Mann spielt auf der Bühne auf einem Instrument . <eos>
Prediction:  A A man is a a a on a couch .
Target:  <sos> A man playing an instrument on stage . <eos>

 
source:  <sos> Ein Mann und zwei Kinder spielen für eine Gruppe von Zuschauern Musik . <eos>
Prediction:  A A man and a woman are in the grass . <eos>
Target:  <sos> A man and two young children play music for a group of onlookers . <eos>

 
source:  <sos> Archäologen graben in der Erde nach etwas . <eos>
Prediction:  A A little boy in a a a . <eos> <eos>
Target:  <sos> Archaeologists are on the ground digging for something . <eos>

 
source:  <sos> Ein Mann liegt in der Mitte einer schmutzigen Gasse . <eos>
Prediction:  A A man man is a a a a . . <eos>
Target:  <sos> A man is lying down in the middle of a dirty alley . <eos>

 
source:  <sos> Schwerarbeiter verladen an einem regnerischen Tag Kies . <eos>
Prediction:  A A little boy in a a a . <eos> <eos>
Target:  <sos> Hard labor workers moving gravel on a r

source:  <sos> Eine Frau geht an einer Gruppe von drei Männern vorbei . <eos>
Prediction:  A A woman is a a a a a a a a a . <eos>
Target:  <sos> A woman is walking by a group of three men . <eos>

 
source:  <sos> Fans schauen zu , wie Spieler zwischen den Sessions bei einem Football-Spiel ausruhen . <eos>
Prediction:  Two Two people are in a a in a gym . <eos>
Target:  <sos> Fans watch as players rest between sessions in a football game . <eos>

 
source:  <sos> Zwei Männer und eine Frau unterhalten sich an einem Tisch . <eos>
Prediction:  Two Two men and women are yoga in a gym . <eos>
Target:  <sos> Two men and a woman interact at a table . <eos>

 
145.29600753635168
[100][loss: 3.85][pp:46.84]
[200][loss: 3.70][pp:40.61]
[300][loss: 3.63][pp:37.67]
[400][loss: 3.73][pp:41.72]
[500][loss: 3.77][pp:43.57]
[600][loss: 3.77][pp:43.28]
[700][loss: 3.76][pp:42.95]
[800][loss: 3.74][pp:42.08]
[900][loss: 3.77][pp:43.42]
source:  <sos> Ein Mann in gelber Kleidung sieht einigen Männern bei

[300][loss: 3.69][pp:40.21]
[400][loss: 3.73][pp:41.51]
[500][loss: 3.69][pp:39.89]
[600][loss: 3.68][pp:39.73]
[700][loss: 3.63][pp:37.57]
[800][loss: 3.71][pp:40.72]
[900][loss: 3.74][pp:41.98]
source:  <sos> Ein Junge mit einer blauen Mütze zeigt auf eine Schüssel voller Kork . <eos>
Prediction:  A A man in a yellow shirt is a a . <eos>
Target:  <sos> A boy with a blue hat points towards a bowl full of cork . <eos>

 
source:  <sos> Zwei Männer in verschieden farbigen Trikots , spielen Hockey . <eos>
Prediction:  A Two men in a in a of a . <eos>
Target:  <sos> Two men in different colored jerseys play hockey . <eos>

 
source:  <sos> Der Hund rennt im Gras neben dem Wasser . <eos>
Prediction:  A Two dogs dogs are running through the snow . <eos>
Target:  <sos> The dog is running in the grass next to the water . <eos>

 
source:  <sos> Ein Mann liegt im Schnee und ein Mädchen steht neben ihm . <eos>
Prediction:  A A man and a woman are in a a . <eos>
Target:  <sos> A man lays in the 

source:  <sos> Ein Mann und zwei kleine Kinder entspannen sich auf einem Zeltplatz und sitzen essend vor einem Feuer . <eos>
Prediction:  A A man and a a and a a and a a on a bench . <eos>
Target:  <sos> A man and two small children relaxing at a campsite , eating in front of a fire . <eos>

 
source:  <sos> Eine Frau spielt in einem leuchtend orangen Outfit Tennis . <eos>
Prediction:  A A woman in a blue dress is a a .
Target:  <sos> Woman playing tennis in a bright orange outfit . <eos>

 
source:  <sos> Ein Mann wirbt für verschiedene Arten von Meeresfrüchten auf dem Stock , auch für Seesterne . <eos>
Prediction:  A A man is a a his bike on the street . <eos>
Target:  <sos> A man is showcasing various types of seafood on a stick , including starfish . <eos>

 
source:  <sos> Ein Mann in einem gelben Hemd blickt auf seinen blauen Wäschesack neben einer Waschmaschine . <eos>
Prediction:  A A man in a blue shirt is a a a a a a a a . <eos>
Target:  <sos> Man in yellow shirt looking into

source:  <sos> Eine geschäftige Straße mit vielen Fußgängern . <eos>
Prediction:  A A person is a a a a mountain . <eos>
Target:  <sos> A crowded city street with lots of pedestrians . <eos>

 
source:  <sos> Ein Mann fährt draußen an einer orange Wand mit einem Fahrrad vorbei . <eos>
Prediction:  A A man in a red jacket is on a a . <eos>
Target:  <sos> A man rides a bicycle past an orange wall outside . <eos>

 
source:  <sos> Ein Mädchen mit einem Handtuch neben einem Waschbecken . <eos>
Prediction:  A A little girl in a pink shirt is a a . <eos>
Target:  <sos> A female child with a towel next to a sink . <eos>

 
source:  <sos> Eine glückliches älteres Paar tanzt miteinander , während eine jüngere Tänzerin zuschaut . <eos>
Prediction:  A A man in a a shirt is clipping a man in a a . <eos>
Target:  <sos> A happy older couple share a dance while a younger dancing woman looks on . <eos>

 
source:  <sos> Eine blonde Frau sitzt an einem Tisch und bietet einer anderen blonden Frau einen 

In [18]:
train(seq2seq, optimizer, train_iter, val_iter, batch_size=32, num_epochs=20)

[100][loss: 3.63][pp:37.77]
[200][loss: 3.71][pp:40.97]
[300][loss: 3.67][pp:39.15]
[400][loss: 3.70][pp:40.56]
[500][loss: 3.70][pp:40.64]
[600][loss: 3.79][pp:44.13]
[700][loss: 3.82][pp:45.74]
[800][loss: 3.76][pp:43.09]
[900][loss: 3.81][pp:45.06]


/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


source:  <sos> Der rot gekleidete Football-Spieler schaut über das Spielfeld . <eos>
Prediction:  A A man in a a shirt is a . <eos>
Target:  <sos> The football player dressed in red looks down field . <eos>

 
source:  <sos> Ein älterer Mann und eine junge Frau lachen gemeinsam am Strand . <eos>
Prediction:  A A man and a woman are sitting on a wall . <eos>
Target:  <sos> An elderly man and a young woman share a laugh on the beach . <eos>

 
source:  <sos> Die Kinder sahen dem jungen Jungen beim Schreiben auf das orangefarbene Papier zu . <eos>
Prediction:  A A man in a white shirt is a a a . <eos>
Target:  <sos> The children watched the young boy write on the orange paper . <eos>

 
source:  <sos> Ein einsamer Fischer überprüft auf seinem Boot die Netze . <eos>
Prediction:  A A man in a on a beach . <eos>
Target:  <sos> A lone fisherman is on his boat checking his net . <eos>

 
source:  <sos> Ein Mann sitzt auf einem Motorrad am Bürgersteig . <eos>
Prediction:  A A man in a white shi

/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  """


[100][loss: 3.87][pp:47.83]
[200][loss: 3.67][pp:39.33]
[300][loss: 3.68][pp:39.81]
[400][loss: 3.65][pp:38.65]
[500][loss: 3.69][pp:39.90]
[600][loss: 3.76][pp:43.03]
[700][loss: 3.63][pp:37.60]
[800][loss: 3.72][pp:41.43]
[900][loss: 3.67][pp:39.42]
source:  <sos> Ein Mann malt vor einer Schaufensterfront , wo seine Gemälde ausgestellt sind . <eos>
Prediction:  A A man in a a a a a a of a a a a a a
Target:  <sos> A man painting in front of a storefront , his paintings are displayed . <eos>

 
source:  <sos> Viele Personen beobachten einen Footballspieler , der einen Ball fängt . <eos>
Prediction:  A A group of people are in a the background . <eos>
Target:  <sos> Many people are witnessing a football player catching the ball <eos>

 
source:  <sos> Vier Asiatinnen posieren für ein Foto . <eos>
Prediction:  A A people are walking on a street . <eos>
Target:  <sos> Four Asian women pose for a picture . <eos>

 
source:  <sos> Ein männlicher und zwei weibliche Wanderer posieren vor eine

source:  <sos> Zwei Männer aus Nahost in weißen Roben spielen auf einer großen Gitarre und einer Bongo . <eos>
Prediction:  Two Two men are a woman reflective clothing and a a jeep in a a a a a a
Target:  <sos> Two middle eastern men dressed in white robes are playing a large guitar and a bongo . <eos>

 
source:  <sos> Ein Mann jongliert neben seinen zwei Freunden mit Tennisbällen . <eos>
Prediction:  A A man is a a a a a . <eos>
Target:  <sos> A man is juggling tennis balls next to his two friends . <eos>

 
source:  <sos> Ein Mann spielt Golf und schlägt den Ball aus einem Sandbunker hinaus . <eos>
Prediction:  A A man and a woman are a a a . <eos>
Target:  <sos> A man is playing golf , striking his ball out of a sand bunker . <eos>

 
source:  <sos> Männer in einem Pulk bei einem Radrennen . <eos>
Prediction:  A People are walking a bus stop station . <eos>
Target:  <sos> Men in a cluster during a bike race <eos>

 
source:  <sos> Der schwarze Hund fängt etwas Grünes mit dem Maul .

source:  <sos> Eine Gruppe von Menschen sitzt draußen . <eos>
Prediction:  A A group of people in a in a
Target:  <sos> A group of people sitting outdoors . <eos>

 
source:  <sos> Eine junge Frau arbeitet hinter der Theke einer Saftbar . <eos>
Prediction:  A A woman in a black shirt is a a . <eos>
Target:  <sos> A young woman works behind the counter at a juice bar . <eos>

 
source:  <sos> Ein Mann in einem weißen Hemd fügt einer kleinen Skulptur Details hinzu . <eos>
Prediction:  A A man man is a a a is playing a a . <eos>
Target:  <sos> A man in a white shirt adds detail to small sculptures . <eos>

 
source:  <sos> Eine Frau und ein Mann posieren in Groucho-Marx-Kostümen . <eos>
Prediction:  A A man and woman are standing in front of a toll .
Target:  <sos> A woman and a man pose with Groucho Marx disguises . <eos>

 
source:  <sos> Ein kleiner Junge mit einer lilafarbenen Jacke sitzt angeschnallt in seinem Autositz . <eos>
Prediction:  A A boy in a red shirt is a a a a . <eos>
Ta

source:  <sos> Eine in Rot und Schwarz gekleidete Person außerhalb einer Wand mit Rissen . <eos>
Prediction:  A A woman is a a and a man in a a a . <eos>
Target:  <sos> A person dressed in red and black outside a cracked wall . <eos>

 
source:  <sos> Eine Frau in einem roten Sweatshirt und einer weißen Schürze wäscht eine große Pfanne in einer Spüle . <eos>
Prediction:  A A woman in a black shirt and jeans is a a a a a . <eos> . <eos>
Target:  <sos> A woman dressed in a red sweatshirt , wearing a white apron , and washing a large pan in a sink . <eos>

 
source:  <sos> Eine Frau mit einem gepunkteten Regenschirm geht durch den Schnee . <eos>
Prediction:  A A woman is a woman is a her cellphone while her lap . <eos>
Target:  <sos> A woman is walking in the snow with a polka dot umbrella . <eos>

 
145.3639044612646
[100][loss: 3.78][pp:43.91]
[200][loss: 3.58][pp:35.79]
[300][loss: 3.66][pp:38.81]
[400][loss: 3.69][pp:40.22]
[500][loss: 3.67][pp:39.16]
[600][loss: 3.69][pp:40.20]
[700]

source:  <sos> Ein Mann mit grüner Jacke und Strickmütze auf einem Markt im Freien sortiert Tomaten . <eos>
Prediction:  A A man in a a shirt and a a a a a a a . <eos>
Target:  <sos> A man wearing a green jacket and knit hat is sorting tomatoes at an outdoor market . <eos>

 
146.70414175093174
[100][loss: 3.73][pp:41.76]
[200][loss: 3.59][pp:36.27]
[300][loss: 3.62][pp:37.16]
[400][loss: 3.70][pp:40.30]
[500][loss: 3.58][pp:35.91]
[600][loss: 3.62][pp:37.37]
[700][loss: 3.66][pp:39.02]
[800][loss: 3.73][pp:41.82]
[900][loss: 3.74][pp:42.21]
source:  <sos> Ein Mann trägt ein Möbelstück aus einem Laster . <eos>
Prediction:  A A dog is a a a . <eos>
Target:  <sos> A man is walking out a truck while carrying furniture . <eos>

 
source:  <sos> Die Hockeyspieler bewegen sich schnell in Richtung des Pucks . <eos>
Prediction:  A A woman in a a dress is interviewing a a
Target:  <sos> The hockey players move quickly towards the puck . <eos>

 
source:  <sos> Ein kleiner Junge bearbeitet den B

[100][loss: 3.80][pp:44.82]
[200][loss: 3.50][pp:33.16]
[300][loss: 3.59][pp:36.28]
[400][loss: 3.55][pp:34.74]
[500][loss: 3.66][pp:38.76]
[600][loss: 3.65][pp:38.48]
[700][loss: 3.65][pp:38.49]
[800][loss: 3.69][pp:40.01]
[900][loss: 3.72][pp:41.19]
source:  <sos> Leute stehen draußen und beobachten , wie Motorräder eine Straße entlang fahren . <eos>
Prediction:  A A group of people are sitting and eating . <eos>
Target:  <sos> People stand outside watching motorcycles drive down a road . <eos>

 
source:  <sos> Ein kleiner Junge hält sich an einer Seilrutsche fest und fährt darauf einen Hügel hinunter . <eos>
Prediction:  A A girl in a and sweatsuit in a grassy field path . <eos>
Target:  <sos> A young boy is holding on and riding a zip line down a hill . <eos>

 
source:  <sos> Eine Frau in einer U-Bahn schläft ein . <eos>
Prediction:  A A woman in a a jacket is a a .
Target:  <sos> A woman on a subway is falling asleep . <eos>

 
source:  <sos> Ein Paar sitzt auf einem schrägen Ba

In [19]:
train(seq2seq, optimizer, train_iter, val_iter, batch_size=32, num_epochs=20)

[100][loss: 3.59][pp:36.14]
[200][loss: 3.55][pp:34.64]
[300][loss: 3.64][pp:37.97]
[400][loss: 3.56][pp:35.07]
[500][loss: 3.57][pp:35.66]
[600][loss: 3.63][pp:37.57]
[700][loss: 3.62][pp:37.27]
[800][loss: 3.59][pp:36.11]
[900][loss: 3.62][pp:37.48]


/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


source:  <sos> Ein dicker Mann in einem lila Hemd isst einen Kartoffelchip . <eos>
Prediction:  A A man in a blue shirt is a a a . <eos> .
Target:  <sos> A fat man in a purple shirt eating a potato chip . <eos>

 
source:  <sos> Ein älterer kahlköpfiger Mann mit Bart im hellen Hemd sitzt auf der Straße und raucht . <eos>
Prediction:  A A man in a blue and white striped shirt is sitting on a bench . <eos>
Target:  <sos> An older bold man with a beard in the bright shirt is sitting and smoking on the street . <eos>

 
source:  <sos> Drei männliche Freunde in Shorts stehen zusammen und unterhalten sich . <eos>
Prediction:  A Three women are in in the snow and a . <eos> . <eos>
Target:  <sos> Three male friends in shorts stand together having a conversation . <eos>

 
source:  <sos> Ein Mann verwendet einen Mopp , während zwei andere Männer aus dem Hintergrund zusehen . <eos>
Prediction:  A A man in a blue shirt is playing a piano . <eos>
Target:  <sos> One man is using a mop , while two o

/home/austin/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  """


146.00668227672577
[100][loss: 3.78][pp:43.90]
[200][loss: 3.63][pp:37.86]
[300][loss: 3.54][pp:34.61]
[400][loss: 3.61][pp:37.04]
[500][loss: 3.74][pp:42.16]
[600][loss: 3.60][pp:36.76]
[700][loss: 3.67][pp:39.23]
[800][loss: 3.66][pp:39.04]
[900][loss: 3.70][pp:40.33]
source:  <sos> Ein Mann in einem gestreiften Oberteil , mit Weste , Krawatte und Hut schenkt sich Bowle auf einer großen Schüssel ein . <eos>
Prediction:  A A man in a blue shirt and a shorts is a a a a a a a a a a a a a a a a a a .
Target:  <sos> A man in a striped shirt , vest , tie , and hat is serving himself punch from a large bowl . <eos>

 
source:  <sos> Eine Frau mit einem blauen Hemd und Kreolen . <eos>
Prediction:  A A woman in a red shirt and a woman is playing a
Target:  <sos> A woman with a blue shirt and hoop earrings . <eos>

 
source:  <sos> Ein Mann auf einem grünen Stuhl am Flughafen . <eos>
Prediction:  A A man is on a rock on a . <eos>
Target:  <sos> A man sitting on a green chair in an airport . <e

source:  <sos> Der Reiter wirft für ein besseres Gleichgewicht die Hand in die Luft , während das braune Pferd buckelt . <eos>
Prediction:  A The boy fed a foreign man , with a knife and the other . <eos>
Target:  <sos> The rider throws his hand up for balance as the brown horse bucks . <eos>

 
source:  <sos> Eine Gruppe tanzt in einem orientalischen Restaurant . <eos>
Prediction:  A A woman in a a dress is grains . <eos>
Target:  <sos> A group of people dance at a oriental restaurant . <eos>

 
source:  <sos> Ein älterer Mann mit einem Gehgestell geht vor einer bunten Mauer entlang , auf der vielerlei passiert . <eos>
Prediction:  A A man in a a a a a a a a a a a a a a shirt is a a . <eos>
Target:  <sos> A older man with a walker walking in front of a colorful wall , with multiple things going on it . <eos>

 
source:  <sos> Ein junger Mann steht mit verschränkten Armen im Freien vor elektronischen Geräten , während ein anderer Mann im Hintergrund Kopfhörer trägt . <eos>
Prediction: 

source:  <sos> Mehrere junge Menschen spielen ein Brettspiel auf einer großen Decke in einem grasbewachsenen Feld . <eos>
Prediction:  A A band are on a bench in a park bench . <eos>
Target:  <sos> Several young people play a board game using a large blanket in a grassy field . <eos>

 
source:  <sos> Zwei dunkelhäutige Männer in schwarzen Mänteln werden von einer Frau betrachtet , als sie die Straße entlanggehen . <eos>
Prediction:  A Two men in a shirts are in a maybe in a martial arts match . <eos>
Target:  <sos> Two black men in black coats walk down the street with women looking at them . <eos>

 
source:  <sos> Eine Frau mit Sonnenbrille telefoniert mit ihrem Mobiltelefon . <eos>
Prediction:  A A woman in a black shirt is a a carousel . <eos>
Target:  <sos> A female with sunglasses is talking on her cellphone . <eos>

 
source:  <sos> Eine Frau in einem roten Kleid bedeckt die Augen einer anderen Frau . <eos>
Prediction:  A A woman in a blue shirt is a a a a a a .
Target:  <sos> 

source:  <sos> Eine Frau , mehrere Männer und mehrere Kinder sind auf einem Gehsteig , auf dem Erzeugnisse verkauft werden . <eos>
Prediction:  A A woman and a a shirt and khaki pants and a heeled boots . <eos>
Target:  <sos> There is a woman , several men , and several children on a sidewalk where produce is being sold . <eos>

 
source:  <sos> Zwei Männer mit Rucksäcken betrachten Damenunterwäsche in einem Schaufenster . <eos>
Prediction:  A Two men in a shirts are a ceramic tile . <eos>
Target:  <sos> Two men with backpacks on looking at lingerie in a window . <eos>

 
source:  <sos> Personen schauen hoch , als der Himmel von etwas erleuchtet wird . <eos>
Prediction:  A A man with a red shirt is a . <eos>
Target:  <sos> People watch as something lights up the sky . <eos>

 
148.66607685387135
[100][loss: 7.45][pp:1722.62]
[200][loss: 3.67][pp:39.44]
[300][loss: 3.70][pp:40.33]
[400][loss: 3.66][pp:38.67]
[500][loss: 3.62][pp:37.32]
[600][loss: 3.68][pp:39.79]
[700][loss: 3.71][pp:41

source:  <sos> Hunde schlafen in einem asiatischen Land auf einer Betonstraße im Freien . <eos>
Prediction:  A A person is a on the snow with a snow . <eos>
Target:  <sos> Dog sleeps outside on the cemented street in asian country . <eos>

 
source:  <sos> Ein paar Männer reparieren etwas mitten auf einer Straße . <eos>
Prediction:  A A person is on a snowboard in a bike . <eos>
Target:  <sos> Some men are fixing something in the middle of a street . <eos>

 
149.29469987004995
[100][loss: 3.85][pp:46.97]
[200][loss: 3.61][pp:36.86]
[300][loss: 3.51][pp:33.49]
[400][loss: 3.58][pp:35.83]
[500][loss: 3.66][pp:38.69]
[600][loss: 3.64][pp:37.97]
[700][loss: 3.70][pp:40.45]
[800][loss: 3.60][pp:36.59]
[900][loss: 3.67][pp:39.27]
source:  <sos> Ein Mann in orangefarbener Sicherheitsweste wendet sich in einem Wald an mehrere andere Männer . <eos>
Prediction:  A A man in a black shirt is a a . <eos>
Target:  <sos> A man in an orange safety vest is addressing several other men in a forest . <e

[100][loss: 3.75][pp:42.55]
[200][loss: 3.56][pp:35.11]
[300][loss: 3.65][pp:38.37]
[400][loss: 3.63][pp:37.84]
[500][loss: 3.68][pp:39.73]
[600][loss: 3.67][pp:39.08]
[700][loss: 3.64][pp:38.03]
[800][loss: 3.67][pp:39.10]
[900][loss: 3.71][pp:41.04]
source:  <sos> Eine Frau mit kurzen , dunklen Haaren sitzt auf dem Boden und liest einem kleinen Mädchen in einem weißen Kleid aus einem Buch vor . <eos>
Prediction:  A A woman with a hair and a corsage clasps a man in a a a a a a a a a a a a a a a a a a a a a . shirt . <eos>
Target:  <sos> A woman with short , dark hair is on the floor reading a book to a little girl dressed in a white dress . <eos>

 
source:  <sos> Ein Mann in einem blauen T-Shirt wendet Hamburger und Hotdogs auf einem Grill im Freien . <eos>
Prediction:  A A man in a blue shirt and blue pants and a sock . <eos>
Target:  <sos> Guy in a blue shirt flipping burgers and hotdogs on an outdoor grill . <eos>

 
source:  <sos> Ein Mann und eine Frau sitzen an der Küste und sc